In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score, classification_report, confusion_matrix

# Database connection setup
db_config = {
    'host': 'localhost',
    'database': 'postgres',
    'user': 'postgres',
    'password': 'kaviyam123',
    'port': '5432'
}
connection_string = f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"
engine = create_engine(connection_string)

# Load data from PostgreSQL
query = 'SELECT * FROM public.overall_cleaned_base_and_pr_ef_policyef;'
data = pd.read_sql(query, con=engine)

In [2]:
selected_columns = ['policy no', 'renewal type', 'product name', 'product name 2',  'biztype', 'policy end date', 'policy start date', 
 'age', 'manufacturer/make', 'model', 'variant', 'vehicle segment', 'fuel type', 'rto location', 'vehicle idv', 'ncb amount', 'Cleaned Reg no', 
 'before gst add-on gwp', 'total od premium', 'total tp premium', 'gst', 'total premium payable', 
 'ncb % previous year', 'applicable discount with ncb', 'Cleaned Branch Name 2', 'Cleaned State2', 'Cleaned Zone 2', 'tie up',
 'Number of claims', 'approved', 'denied', 'corrected_name', 'customerid', 'Policy Status', 'Policy Tenure', 'Customer Tenure', 'New Customers', 'Claim Happaned/Not', 
 'Renewal Rate Status', 'withdrawn', 'chassis_engine_key', 'policy_wise_purchase']

data = data[selected_columns]

# Convert Policy End Date to datetime
data['policy end date'] = pd.to_datetime(data['policy end date'], errors='coerce')

# Separate Open Customers (August to October 2024)
open_customers = data[
    (data['Policy Status'].isin(['Renewed', 'Not Renewed'])) &
    (data['policy end date'].dt.year == 2024) &
    (data['policy end date'].dt.month.isin([9, 10, 11, 12]))
].copy()

# Filter the main dataset for customers whose Policy End Date is <= July 2024
data = data[
    (data['Policy Status'].isin(['Renewed', 'Not Renewed'])) &
    (data['policy end date'] <= pd.to_datetime('2024-08-31'))
]

# Map Policy Status to binary
data['Policy Status'] = data['Policy Status'].apply(lambda x: 1 if x == 'Not Renewed' else 0)

# Handle missing values
for column in data.columns:
    if data[column].dtype == 'object':
        data[column] = data[column].fillna('missing')
    else:
        data[column] = data[column].fillna(0)

# Extract year, month, and day from date columns
date_columns = ['policy start date', 'policy end date']
for col in date_columns:
    data[col] = pd.to_datetime(data[col], errors='coerce')

# Extract date features
for col in date_columns:
    data[f'{col}_YEAR'] = data[col].dt.year
    data[f'{col}_MONTH'] = data[col].dt.month
    data[f'{col}_DAY'] = data[col].dt.day

# Drop original date columns
data.drop(columns=date_columns, inplace=True)

# Separate features and target variable for training
features = [col for col in data.columns if col != 'Policy Status']
X = data[features]
y = data['Policy Status']

from imblearn.over_sampling import RandomOverSampler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, log_loss, roc_curve
import matplotlib.pyplot as plt

# Initialize RandomOverSampler
ros = RandomOverSampler(random_state=42)

# Apply Random Oversampling to the training data
X, y = ros.fit_resample(X, y)

# Apply the same transformations to open_customers
for col in date_columns:
    open_customers[col] = pd.to_datetime(open_customers[col], errors='coerce')

for col in date_columns:
    open_customers[f'{col}_YEAR'] = open_customers[col].dt.year
    open_customers[f'{col}_MONTH'] = open_customers[col].dt.month
    open_customers[f'{col}_DAY'] = open_customers[col].dt.day

open_customers.drop(columns=date_columns, inplace=True)

for column in open_customers.columns:
    if open_customers[column].dtype == 'object':
        open_customers[column] = open_customers[column].fillna('missing')
    else:
        open_customers[column] = open_customers[column].fillna(0)

open_customers_without_encoded = open_customers.copy()

In [3]:
# Apply label encoding to categorical features
label_encoders = {}
for column in X.columns:
    if X[column].dtype == 'object':
        label_encoder = LabelEncoder()
        X[column] = label_encoder.fit_transform(X[column].astype(str))
        label_encoders[column] = label_encoder
        
        mapping_dict = {label: i for i, label in enumerate(label_encoder.classes_)}
        next_unique_value = max(mapping_dict.values()) + 1  

        def encode_test_value(value):
            return mapping_dict.get(value, next_unique_value)

        open_customers[column] = open_customers[column].apply(encode_test_value)

In [4]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(
    depth=10,                    
    learning_rate=0.1,            
    iterations=500,               
    random_seed=42,               
    verbose=0                     
)


model.fit(X, y)

# Predict using encoded data
X_open_customers = open_customers[features]
y_open_pred = model.predict(X_open_customers)
y_open_pred_proba = model.predict_proba(X_open_customers)[:, 1]  

# Store prediction results in unencoded data
open_customers_without_encoded['Predicted Status'] = np.where(y_open_pred == 1, 'Not Renewed', 'Renewed')
open_customers_without_encoded['Churn Probability'] = y_open_pred_proba

print(f"Predicted Renewed: {(y_open_pred == 0).sum()}")
print(f"Predicted Not Renewed: {(y_open_pred == 1).sum()}")

# Evaluate model on training data
y_pred = model.predict(X)
y_pred_proba = model.predict_proba(X)[:, 1]

#Evaluate the model on training data
train_accuracy = accuracy_score(y, y_pred)
train_log_loss = log_loss(y, y_pred_proba)
train_roc_auc = roc_auc_score(y, y_pred_proba)
train_report = classification_report(y, y_pred)

conf_matrix_train = confusion_matrix(y, y_pred)
class_0_accuracy_train = conf_matrix_train[0, 0] / conf_matrix_train[0].sum()
class_1_accuracy_train = conf_matrix_train[1, 1] / conf_matrix_train[1].sum()

#Print the metrics
print(f"Train Accuracy: {train_accuracy}")
print(f"Train Log Loss: {train_log_loss}")
print(f"Train ROC AUC: {train_roc_auc}")
print(f"Train Classification Report:\n{train_report}")
print(f"Class 0 Train Accuracy: {class_0_accuracy_train}")
print(f"Class 1 Train Accuracy: {class_1_accuracy_train}")

Predicted Renewed: 48642
Predicted Not Renewed: 217079
Train Accuracy: 0.7523645565972796
Train Log Loss: 0.501083515447137
Train ROC AUC: 0.8329782910106387
Train Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.79      0.76    749612
           1       0.77      0.72      0.74    749612

    accuracy                           0.75   1499224
   macro avg       0.75      0.75      0.75   1499224
weighted avg       0.75      0.75      0.75   1499224

Class 0 Train Accuracy: 0.7871058627663378
Class 1 Train Accuracy: 0.7176232504282215


In [5]:
# Save predictions
open_customers_without_encoded.to_csv('SOND_predictions_cat_check.csv', index=False)

In [6]:
from catboost import CatBoostClassifier
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss, classification_report, confusion_matrix, roc_curve

# Define the models
model_1 = CatBoostClassifier(
    depth=6, learning_rate=0.1, iterations=100, 
    random_seed=42, verbose=0)

model_2 = CatBoostClassifier(
    depth=10, learning_rate=0.1, iterations=500, 
    random_seed=42, verbose=0)

# Fit both models
model_1.fit(X, y)
model_2.fit(X, y)

# Predict probabilities
proba_1 = model_1.predict_proba(X)
proba_2 = model_2.predict_proba(X)

# Adjust decision threshold to favor Class 0
threshold_0 = 0.60  # Increase bias towards Class 0
threshold_1 = 0.50  # Lower threshold for Class 1

y_pred = np.where(proba_1[:, 0] > threshold_0, 0, 
                  np.where(proba_2[:, 1] > threshold_1, 1, 0))
y_pred_proba = np.maximum(proba_1[:, 0], proba_2[:, 1])

# Store prediction results in open customer data
X_open_customers = open_customers[features]
proba_open_1 = model_1.predict_proba(X_open_customers)
proba_open_2 = model_2.predict_proba(X_open_customers)

y_open_pred = np.where(proba_open_1[:, 0] > threshold_0, 0, 
                       np.where(proba_open_2[:, 1] > threshold_1, 1, 0))
y_open_pred_proba = np.maximum(proba_open_1[:, 0], proba_open_2[:, 1])

# Store prediction results in unencoded data
open_customers_without_encoded['Predicted Status'] = np.where(y_open_pred == 1, 'Not Renewed', 'Renewed')
open_customers_without_encoded['Churn Probability'] = y_open_pred_proba

print(f"Predicted Renewed: {(y_open_pred == 0).sum()}")
print(f"Predicted Not Renewed: {(y_open_pred == 1).sum()}")

# Evaluate model on training data
train_accuracy = accuracy_score(y, y_pred)
train_log_loss = log_loss(y, y_pred_proba)
train_roc_auc = roc_auc_score(y, y_pred_proba)
train_report = classification_report(y, y_pred)

conf_matrix_train = confusion_matrix(y, y_pred)
class_0_accuracy_train = conf_matrix_train[0, 0] / conf_matrix_train[0].sum()
class_1_accuracy_train = conf_matrix_train[1, 1] / conf_matrix_train[1].sum()

# Print the metrics
print(f"Train Accuracy: {train_accuracy}")
print(f"Train Log Loss: {train_log_loss}")
print(f"Train ROC AUC: {train_roc_auc}")
print(f"Train Classification Report:\n{train_report}")
print(f"Class 0 Train Accuracy: {class_0_accuracy_train}")
print(f"Class 1 Train Accuracy: {class_1_accuracy_train}")


Predicted Renewed: 97472
Predicted Not Renewed: 168249
Train Accuracy: 0.7471511928837852
Train Log Loss: 0.800480203816853
Train ROC AUC: 0.6157244358285027
Train Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.79      0.76    749612
           1       0.77      0.70      0.74    749612

    accuracy                           0.75   1499224
   macro avg       0.75      0.75      0.75   1499224
weighted avg       0.75      0.75      0.75   1499224

Class 0 Train Accuracy: 0.7919843865893289
Class 1 Train Accuracy: 0.7023179991782416


In [7]:
# Save predictions
open_customers_without_encoded.to_csv('SOND_predictions_cat_combined_check.csv', index=False)